In [1]:
import torch
import numpy as np
from qdrant_client import QdrantClient
from typing import List, Dict, Any
import sys
import os
import json

sys.path.append("./src")

from retriever import RetrievalService

os.makedirs("artifacts", exist_ok=True)
with open("artifacts/user_to_idx.json", "w") as f:
    json.dump({"u123": 1, "u456": 2}, f) # u999 será desconhecido

# Atualizado para apontar para o checkpoint correto, host do Qdrant e nome da coleção
service = RetrievalService(
    model_path="data/two_tower_model.ckpt", 
    artifacts_path="artifacts",
    qdrant_host="qdrant",
    collection_name="hm_items"
)

⚡ Inicializando Retrieval Service em cuda...
   └── Modelo carregado de data/two_tower_model.ckpt
   └── Artefatos carregados (Total usuários conhecidos: 2)


In [2]:
# Teste 1: Usuário Conhecido
print("\n--- Teste: Usuário Conhecido (u123) ---")
results = service.retrieve_candidates("u123", {"age": 30}, k=3)
print(f"Recuperados: {len(results)} itens")


--- Teste: Usuário Conhecido (u123) ---
Recuperados: 3 itens


In [3]:
# Teste 2: Cold Start (u999)
print("\n--- Teste: Cold Start (u999) ---")
# Aqui o preprocess deve cair no índice 0 (<UNK>) e ainda assim retornar itens (mesmo que genéricos)
results = service.retrieve_candidates("u999", {"age": 20}, k=3) 
print(f"Recuperados: {len(results)} itens (Fallback funcionando)")


--- Teste: Cold Start (u999) ---
Recuperados: 3 itens (Fallback funcionando)
